# 🚀 Complete AI Scheduler Workflow

This notebook implements a complete end-to-end AI scheduling system with automatic calendar booking:

1. **User Input** - Natural language scheduling requests
2. **AI Parsing** - Extract structured data using Keywords AI (prompt ID: 01ce06xxxxxx)  
3. **Calendar Availability** - Find available 30-minute slots in Google Calendar
4. **AI Recommendation** - Get optimal time slot recommendation (prompt ID: 7e2facxxxxxx)
5. **Automatic Booking** - Create calendar event with proper timezone handling

**Complete automation:** From "Schedule a meeting tomorrow at 2pm" to a booked calendar event!


## Step 1: Setup & User Input

We start by importing required libraries, loading the Keywords AI API key from the `.env` file, and defining the user's natural language scheduling request.

**What this does:**
- Loads your Keywords AI API key from `KEYWORDSAI_API_KEY` in `.env`
- Sets up the user input string (you can modify this for testing)
- Displays the setup confirmation


In [15]:
import requests
import json
import os
from datetime import datetime, timedelta, date

# Load API key from .env file
try:
    from dotenv import load_dotenv
    load_dotenv()
except ImportError:
    print("Installing python-dotenv...")
    import subprocess
    subprocess.check_call(["pip", "install", "python-dotenv"])
    from dotenv import load_dotenv
    load_dotenv()

# Get Keywords AI API key from .env
KEYWORDS_AI_API_KEY = os.getenv('KEYWORDSAI_API_KEY')

# Define the user input string
user_input = "Schedule a design review meeting tomorrow at 2pm for 1 hour"

print("✅ Setup complete!")
print(f"💬 User input: '{user_input}'")
print(f"🔑 API key loaded: {KEYWORDS_AI_API_KEY[:8]}...{KEYWORDS_AI_API_KEY[-4:]}")


✅ Setup complete!
💬 User input: 'Schedule a design review meeting tomorrow at 2pm for 1 hour'
🔑 API key loaded: iEolcm0o...e3B9


## Step 2: AI Parsing with Keywords AI

This step uses Keywords AI prompt "01ce06" to intelligently parse natural language scheduling requests into structured JSON data.

**Context Variables Provided:**
- `current_date` - Today's date (YYYY-MM-DD format)
- `current_time` - Current time (HH:MM format)  
- `time_zone` - User's detected timezone
- `user_request` - The natural language input to parse

**Expected JSON Output:**
```json
{
  "hasScheduleRequest": true/false,
  "date": "2025-07-02", 
  "time": "14:00",
  "duration": 60,
  "title": "Design Review Meeting",
  "description": "Meeting description"
}
```

**What this accomplishes:** Converts "Schedule a design review tomorrow at 2pm for 1 hour" into structured data the system can work with.


In [16]:
# Get current date/time context
now = datetime.now()
current_date = now.strftime("%Y-%m-%d")
current_time = now.strftime("%H:%M")
time_zone = str(now.astimezone().tzinfo)

print(f"📅 Current context:")
print(f"   Date: {current_date}")
print(f"   Time: {current_time}")
print(f"   Timezone: {time_zone}")

# Keywords AI API call
url = "https://api.keywordsai.co/chat/completions"

headers = {
    "Content-Type": "application/json",
    "Authorization": f"Bearer {KEYWORDS_AI_API_KEY}",
}

# Use prompt ID 01ce06 with variables (no response_format override)
data = {
    "prompt": {
        "prompt_id": "01ce06",
        "variables": {
            "current_date": current_date,
            "current_time": current_time,
            "time_zone": time_zone,
            "user_request": user_input
        }
    }
}

print("\n🤖 Calling Keywords AI...")
response = requests.post(url, headers=headers, json=data)

if response.status_code == 200:
    result = response.json()
    ai_response = result['choices'][0]['message']['content']
    print("✅ AI Response received!")
    print(f"📋 Raw response: {ai_response}")
    
    # Parse the JSON response
    try:
        schedule_data = json.loads(ai_response)
        print("\n✅ Parsed scheduling data:")
        for key, value in schedule_data.items():
            print(f"   {key}: {value}")
    except json.JSONDecodeError as e:
        print(f"❌ Error parsing JSON: {e}")
        schedule_data = None
else:
    print(f"❌ API Error: {response.status_code} - {response.text}")
    schedule_data = None


📅 Current context:
   Date: 2025-07-02
   Time: 14:18
   Timezone: Pacific Daylight Time

🤖 Calling Keywords AI...
✅ AI Response received!
📋 Raw response: {
"hasScheduleRequest": true,
"date": "2025-07-03",
"time": "14:00",
"duration": 60,
"title": "Design Review Meeting",
"description": "A meeting to review design work."
}

✅ Parsed scheduling data:
   hasScheduleRequest: True
   date: 2025-07-03
   time: 14:00
   duration: 60
   title: Design Review Meeting
   description: A meeting to review design work.


## Step 3: Google Calendar Availability Check

Using the structured data from Step 2, we now check Google Calendar for available time slots on the requested date.

**What this step does:**
- Loads Google Calendar credentials from `google_tokens.json`
- Queries existing events for the target date (9 AM - 5 PM work hours)
- Finds available slots in **30-minute intervals** that fit the requested duration
- Shows busy times and conflicts
- Returns a list of all available time slots

**Key Features:**
- **30-minute precision**: Checks every 30 minutes for flexible scheduling
- **Conflict detection**: Avoids overlapping with existing events  
- **Duration-aware**: Only shows slots that fit the full meeting duration
- **Business hours**: Searches within 9 AM to 5 PM by default

**Output:** List of available times like "09:00 AM, 09:30 AM, 10:00 AM..." that can accommodate the meeting.


In [17]:
# Import Google Calendar libraries
from google.auth.transport.requests import Request
from google.oauth2.credentials import Credentials
from googleapiclient.discovery import build

# Load Google Calendar credentials
TOKEN_FILE = 'google_tokens.json'

print("🔑 Loading Google Calendar credentials...")
if os.path.exists(TOKEN_FILE):
    credentials = Credentials.from_authorized_user_file(TOKEN_FILE)
    
    # Refresh tokens if expired
    if credentials.expired and credentials.refresh_token:
        print("🔄 Refreshing expired tokens...")
        credentials.refresh(Request())
        with open(TOKEN_FILE, 'w') as token:
            token.write(credentials.to_json())
    
    # Create Calendar service
    calendar_service = build('calendar', 'v3', credentials=credentials)
    print("✅ Google Calendar service created!")
else:
    print("❌ google_tokens.json not found! Please run google_auth_setup.ipynb first.")
    calendar_service = None


🔑 Loading Google Calendar credentials...
✅ Google Calendar service created!


In [18]:
# Check calendar availability using the AI-parsed data
if schedule_data and calendar_service and schedule_data.get('hasScheduleRequest'):
    
    print("📅 Checking calendar availability...")
    
    # Extract data from AI response
    target_date_str = schedule_data.get('date')
    duration_minutes = schedule_data.get('duration', 60)
    title = schedule_data.get('title', 'Meeting')
    
    print(f"🎯 Target date: {target_date_str}")
    print(f"⏱️  Duration: {duration_minutes} minutes")
    print(f"📝 Title: {title}")
    
    # Convert date string to datetime object
    target_date = datetime.strptime(target_date_str, '%Y-%m-%d').date()
    
    # Set work hours (9 AM to 5 PM)
    work_start = 9
    work_end = 17
    duration_hours = duration_minutes / 60
    
    # Get events for the target date
    start_datetime = datetime.combine(target_date, datetime.min.time().replace(hour=work_start))
    end_datetime = datetime.combine(target_date, datetime.min.time().replace(hour=work_end))
    
    print(f"\n🔍 Checking {target_date} from {work_start}:00 to {work_end}:00...")
    
    # Query Google Calendar for existing events (improved detection)
    # Use broader time range to catch events that might span across the day
    extended_start = datetime.combine(target_date, datetime.min.time())
    extended_end = datetime.combine(target_date, datetime.max.time())
    
    print(f"🔍 Querying calendar from {extended_start.isoformat()} to {extended_end.isoformat()}")
    
    events_result = calendar_service.events().list(
        calendarId='primary',
        timeMin=extended_start.isoformat() + 'Z',
        timeMax=extended_end.isoformat() + 'Z',
        singleEvents=True,
        orderBy='startTime',
        showDeleted=False
    ).execute()
    
    events = events_result.get('items', [])
    print(f"📋 Found {len(events)} existing events on {target_date}")
    
    # Show existing events with more detail
    if events:
        print("🚫 Busy times:")
        for event in events:
            start_time = event['start'].get('dateTime', event['start'].get('date'))
            end_time = event['end'].get('dateTime', event['end'].get('date'))
            summary = event.get('summary', 'No title')
            print(f"   • {summary}")
            print(f"     📅 {start_time} → {end_time}")
    else:
        print("✅ No existing events found - calendar is free!")
    
    # Find available slots with dynamic duration
    slot_duration_text = f"{duration_minutes} minute" if duration_minutes == 1 else f"{duration_minutes} minutes"
    print(f"\n🕐 Finding available {slot_duration_text} slots (checking every 30 minutes)...")
    
    # Extract busy times (improved to handle both datetime and date-only events)
    busy_times = []
    for event in events:
        event_start = event['start'].get('dateTime') or event['start'].get('date')
        event_end = event['end'].get('dateTime') or event['end'].get('date')
        
        if event_start and event_end:
            try:
                # Handle datetime events
                if 'T' in event_start:
                    start = datetime.fromisoformat(event_start.replace('Z', '+00:00')).replace(tzinfo=None)
                    end = datetime.fromisoformat(event_end.replace('Z', '+00:00')).replace(tzinfo=None)
                # Handle all-day events (date-only)
                else:
                    start = datetime.strptime(event_start, '%Y-%m-%d')
                    end = datetime.strptime(event_end, '%Y-%m-%d')
                    # All-day events block the entire day
                    start = start.replace(hour=0, minute=0)
                    end = end.replace(hour=23, minute=59)
                
                busy_times.append((start, end))
                print(f"   🔒 Blocked: {start.strftime('%I:%M %p')} - {end.strftime('%I:%M %p')}")
                
            except ValueError as e:
                print(f"   ⚠️ Could not parse event time: {event_start} - {e}")
    
    print(f"\n📊 Total blocked time periods: {len(busy_times)}")
    
    # Check every 30 minutes for availability (within business hours only)
    available_slots = []
    checked_slots = 0
    current_time = start_datetime
    
    print(f"🔄 Checking slots from {start_datetime.strftime('%I:%M %p')} to {end_datetime.strftime('%I:%M %p')}...")
    
    while current_time + timedelta(hours=duration_hours) <= end_datetime:
        slot_end = current_time + timedelta(hours=duration_hours)
        checked_slots += 1
        
        # Check if this slot conflicts with any busy time
        is_available = True
        conflict_reason = ""
        
        for busy_start, busy_end in busy_times:
            if (current_time < busy_end and slot_end > busy_start):
                is_available = False
                conflict_reason = f"conflicts with event {busy_start.strftime('%I:%M %p')}-{busy_end.strftime('%I:%M %p')}"
                break
        
        if is_available:
            available_slots.append(current_time)
        
        # Move to next 30-minute slot
        current_time += timedelta(minutes=30)
    
    # Display results with summary
    print(f"\n📊 Checked {checked_slots} possible time slots")
    print(f"✅ AVAILABLE TIME SLOTS FOR {slot_duration_text}:")
    
    if available_slots:
        print(f"🎉 Found {len(available_slots)} available slots:")
        for slot in available_slots:
            slot_end_time = slot + timedelta(hours=duration_hours)
            print(f"   ✅ {slot.strftime('%I:%M %p')} - {slot_end_time.strftime('%I:%M %p')}")
    else:
        print(f"❌ No available {slot_duration_text} slots found")
        print("💡 Try a shorter duration or different day")
        
elif not schedule_data:
    print("❌ No schedule data from AI to process")
elif not calendar_service:
    print("❌ Google Calendar service not available")
elif not schedule_data.get('hasScheduleRequest'):
    print("ℹ️  AI determined this is not a scheduling request")

print("\n🏁 Workflow complete!")


📅 Checking calendar availability...
🎯 Target date: 2025-07-03
⏱️  Duration: 60 minutes
📝 Title: Design Review Meeting

🔍 Checking 2025-07-03 from 9:00 to 17:00...
🔍 Querying calendar from 2025-07-03T00:00:00 to 2025-07-03T23:59:59.999999
📋 Found 1 existing events on 2025-07-03
🚫 Busy times:
   • lunch
     📅 2025-07-03T13:00:00-07:00 → 2025-07-03T14:00:00-07:00

🕐 Finding available 60 minutes slots (checking every 30 minutes)...
   🔒 Blocked: 01:00 PM - 02:00 PM

📊 Total blocked time periods: 1
🔄 Checking slots from 09:00 AM to 05:00 PM...

📊 Checked 15 possible time slots
✅ AVAILABLE TIME SLOTS FOR 60 minutes:
🎉 Found 12 available slots:
   ✅ 09:00 AM - 10:00 AM
   ✅ 09:30 AM - 10:30 AM
   ✅ 10:00 AM - 11:00 AM
   ✅ 10:30 AM - 11:30 AM
   ✅ 11:00 AM - 12:00 PM
   ✅ 11:30 AM - 12:30 PM
   ✅ 12:00 PM - 01:00 PM
   ✅ 02:00 PM - 03:00 PM
   ✅ 02:30 PM - 03:30 PM
   ✅ 03:00 PM - 04:00 PM
   ✅ 03:30 PM - 04:30 PM
   ✅ 04:00 PM - 05:00 PM

🏁 Workflow complete!


## Step 4: AI Recommendation & Automatic Calendar Booking

The final step uses AI to select the optimal time slot and automatically creates the calendar event.

**AI Recommendation Process:**
- Uses Keywords AI prompt "7e2fac85550a4d5c98f79f2d708f7820"
- Analyzes user preferences and available slots
- Applies scheduling best practices (morning meetings preferred, avoid lunch hours)
- Returns optimal time with reasoning

**Expected AI Response:**
```json
{
  "suggestedTime": "14:00",
  "duration": "60", 
  "title": "Design Review Meeting",
  "description": "Meeting description",
  "reasoning": "2pm matches user preference and has no conflicts"
}
```

**Automatic Booking Features:**
- **Smart Timezone Handling**: Converts timezone names to proper IANA identifiers  
- **Calendar Integration**: Creates the event directly in Google Calendar
- **Event Details**: Includes title, description, and proper timing
- **Confirmation**: Returns event ID and calendar link

**Result:** A fully booked calendar event with proper timezone, ready for participants to join!


In [19]:
# Step 4: AI Slot Recommendation & Automatic Booking
if available_slots and schedule_data:
    print("\n" + "="*60)
    print("🤖 Step 4: Getting AI recommendation and booking meeting...")
    print("="*60)
    
    # Convert available slots to HH:MM format for AI
    available_times_str = ", ".join([slot.strftime('%H:%M') for slot in available_slots])
    
    print(f"📋 Available slots: {available_times_str}")
    
    # Call Keywords AI with prompt ID for slot recommendation (no response_format override)
    recommendation_data = {
        "prompt": {
            "prompt_id": "7e2fac85550a4d5c98f79f2d708f7820",
            "variables": {
                "user_request": user_input,
                "available_slots": available_times_str
            }
        }
    }
    
    print("\n🤖 Calling Keywords AI for slot recommendation...")
    response = requests.post(url, headers=headers, json=recommendation_data)
    
    if response.status_code == 200:
        result = response.json()
        ai_response = result['choices'][0]['message']['content']
        print("✅ AI recommendation received!")
        print(f"📋 Raw response: {ai_response}")
        
        try:
            # Parse the JSON recommendation
            recommendation = json.loads(ai_response)
            
            print("\n✅ Parsed AI recommendation:")
            print(f"   🕐 Suggested time: {recommendation['suggestedTime']}")
            print(f"   ⏱️  Duration: {recommendation['duration']} minutes") 
            print(f"   📝 Title: {recommendation['title']}")
            print(f"   📄 Description: {recommendation['description']}")
            print(f"   💭 Reasoning: {recommendation['reasoning']}")
            
            # Convert AI recommendation to datetime for Google Calendar
            suggested_time = recommendation['suggestedTime']  # e.g., "12:00"
            recommended_datetime = datetime.strptime(f"{target_date_str} {suggested_time}", '%Y-%m-%d %H:%M')
            end_datetime = recommended_datetime + timedelta(minutes=int(recommendation['duration']))
            
            print(f"\n📅 Booking meeting:")
            print(f"   Start: {recommended_datetime.strftime('%Y-%m-%d %I:%M %p')}")
            print(f"   End: {end_datetime.strftime('%I:%M %p')}")
            
            # Create the Google Calendar event with proper IANA timezone
            def create_calendar_event(service, title, start_time, end_time, description="", user_timezone=None):
                """Create a new calendar event"""
                try:
                    # Convert to proper IANA timezone identifier
                    def get_iana_timezone():
                        """Get proper IANA timezone identifier"""
                        try:
                            # Try to get IANA timezone from system
                            import time
                            import os
                            
                            # For Windows/cross-platform, detect common timezones
                            tz_mapping = {
                                'Pacific Daylight Time': 'America/Los_Angeles',
                                'Pacific Standard Time': 'America/Los_Angeles', 
                                'Eastern Daylight Time': 'America/New_York',
                                'Eastern Standard Time': 'America/New_York',
                                'Central Daylight Time': 'America/Chicago',
                                'Central Standard Time': 'America/Chicago',
                                'Mountain Daylight Time': 'America/Denver',
                                'Mountain Standard Time': 'America/Denver',
                                'PDT': 'America/Los_Angeles',
                                'PST': 'America/Los_Angeles',
                                'EDT': 'America/New_York',
                                'EST': 'America/New_York',
                                'CDT': 'America/Chicago',
                                'CST': 'America/Chicago',
                                'MDT': 'America/Denver',
                                'MST': 'America/Denver'
                            }
                            
                            # Get current timezone info
                            current_tz = str(datetime.now().astimezone().tzinfo)
                            
                            # Try mapping first
                            if current_tz in tz_mapping:
                                return tz_mapping[current_tz]
                            
                            # Try to extract from UTC offset as fallback
                            utc_offset = datetime.now().astimezone().utcoffset().total_seconds() / 3600
                            
                            if utc_offset == -8:  # PST
                                return 'America/Los_Angeles'
                            elif utc_offset == -7:  # PDT or MST
                                return 'America/Los_Angeles'  # Assume Pacific
                            elif utc_offset == -6:  # CST or MDT
                                return 'America/Chicago'
                            elif utc_offset == -5:  # EST or CDT
                                return 'America/New_York'
                            elif utc_offset == -4:  # EDT
                                return 'America/New_York'
                            else:
                                # Default to user's detected timezone or UTC
                                return 'UTC'
                                
                        except:
                            # Ultimate fallback
                            return 'UTC'
                    
                    # Get proper timezone
                    if user_timezone and any(x in user_timezone for x in ['Pacific', 'PDT', 'PST']):
                        iana_timezone = 'America/Los_Angeles'
                    elif user_timezone and any(x in user_timezone for x in ['Eastern', 'EDT', 'EST']):
                        iana_timezone = 'America/New_York'
                    elif user_timezone and any(x in user_timezone for x in ['Central', 'CDT', 'CST']):
                        iana_timezone = 'America/Chicago'
                    elif user_timezone and any(x in user_timezone for x in ['Mountain', 'MDT', 'MST']):
                        iana_timezone = 'America/Denver'
                    else:
                        iana_timezone = get_iana_timezone()
                    
                    print(f"🌍 Using timezone: {user_timezone} → {iana_timezone}")
                    
                    event = {
                        'summary': title,
                        'description': description,
                        'start': {
                            'dateTime': start_time.isoformat(),
                            'timeZone': iana_timezone,
                        },
                        'end': {
                            'dateTime': end_time.isoformat(),
                            'timeZone': iana_timezone,
                        },
                    }
                    
                    created_event = service.events().insert(calendarId='primary', body=event).execute()
                    return created_event
                    
                except Exception as e:
                    print(f"❌ Error creating event: {e}")
                    return None
            
            # Book the meeting automatically
            print("\n🚀 Creating calendar event...")
            created_event = create_calendar_event(
                service=calendar_service,
                title=recommendation['title'],
                start_time=recommended_datetime,
                end_time=end_datetime,
                description=recommendation['description'],
                user_timezone=time_zone  # Pass the timezone from Step 2
            )
            
            if created_event:
                print("✅ Meeting successfully booked!")
                print(f"📋 Event ID: {created_event['id']}")
                print(f"🔗 Event URL: {created_event.get('htmlLink', 'N/A')}")
            else:
                print("❌ Failed to book meeting")
                
        except json.JSONDecodeError as e:
            print(f"❌ Error parsing AI recommendation: {e}")
            
    else:
        print(f"❌ AI API error: {response.status_code} - {response.text}")

else:
    print("⚠️  No available slots to get recommendations for")

print("\n🎉 Complete AI scheduling workflow finished!")



🤖 Step 4: Getting AI recommendation and booking meeting...
📋 Available slots: 09:00, 09:30, 10:00, 10:30, 11:00, 11:30, 12:00, 14:00, 14:30, 15:00, 15:30, 16:00

🤖 Calling Keywords AI for slot recommendation...
✅ AI recommendation received!
📋 Raw response: {
  "suggestedTime": "14:00",
  "duration": "60",
  "title": "Design Review Meeting",
  "description": "A meeting to review design concepts and progress",
  "reasoning": "The user's preferred time of 2pm was selected, which also fits the needed duration of 1 hour. No conflicts were presented at this time."
}

✅ Parsed AI recommendation:
   🕐 Suggested time: 14:00
   ⏱️  Duration: 60 minutes
   📝 Title: Design Review Meeting
   📄 Description: A meeting to review design concepts and progress
   💭 Reasoning: The user's preferred time of 2pm was selected, which also fits the needed duration of 1 hour. No conflicts were presented at this time.

📅 Booking meeting:
   Start: 2025-07-03 02:00 PM
   End: 03:00 PM

🚀 Creating calendar event..

## ✅ Complete Workflow Summary

**🎉 Full End-to-End AI Scheduling System Implemented!**

### **What We Accomplished:**

1. **🗣️ Natural Language Input**: Process requests like "Schedule a design review tomorrow at 2pm for 1 hour"

2. **🤖 AI Parsing**: Keywords AI (prompt 01ce06) extracts:
   - Valid scheduling request detection
   - Target date in YYYY-MM-DD format  
   - Meeting duration in minutes
   - Suggested title and description

3. **📅 Calendar Integration**: Google Calendar API finds:
   - Available 30-minute time slots
   - Conflict detection with existing events
   - Business hours filtering (9 AM - 5 PM)

4. **🎯 AI Recommendation**: Keywords AI (prompt 7e2fac85550a4d5c98f79f2d708f7820) provides:
   - Optimal time slot selection
   - Scheduling best practices application
   - Detailed reasoning for choices

5. **⚡ Automatic Booking**: Google Calendar creates:
   - Properly formatted calendar events
   - Correct timezone handling (IANA format)
   - Event details with title and description
   - Confirmation with event ID and link

### **🚀 Complete Automation Achieved:**
**Input:** `"Schedule a design review tomorrow at 2pm for 1 hour"`  
**Output:** ✅ Booked calendar event at optimal available time!

### **🔧 Customization:**
- Change `user_input` in Step 1 to test different requests
- Modify work hours in Step 3 (default: 9 AM - 5 PM)
- Adjust slot intervals (current: 30 minutes)

**Ready for production use!** 🎊
